# Make an IPS from BgZ resources
- Imports
- Collect all resources in some place
- Set up stuff: server, patient id, patient file, fhir resource files

In [1]:
import os 
import json
import requests
import pandas as pd
from IPython.display import JSON, HTML

zibs = 'zibsr4-examples-json'
# zibsr4 = 'bp-test'
files = os.listdir(zibs)

# base = "http://ips.health:8080/fhir/"
# base = "https://hl7-ips-server.hl7.org/fhir/"
base = "http://hapi.fhir.org/baseR4/"

patientid = "nl-core-Patient-01"
# patientid = "example-bptest"

# Patient file
filename = 'Patient-nl-core-Patient-01--30804749.json'
# filename = 'Patient-nl-core-Patient-test.json'
# filename = 'bp-test/patient-example.json'

- Set patient id
- Collect all resource files, put in dict, show in dataframe for easy inspection.
- For zibsr4 examples I removed pregnancy, since patient was born in 1934
- Made 2 problems inactive to populate History of past problems

In [2]:
resources = []
for file in files:
    if file[0] == '.':
        continue
    with open(os.path.join(zibs, file)) as f:
        jsondata = json.load(f)
        try:
            if "subject" in jsondata:
                patient = jsondata['subject']['reference']
                target = "subject" 
            elif "patient" in jsondata:
                patient = jsondata['patient']['reference']
                target = "patient" 
            else:
                patient = None
            resources.append({"id": jsondata['id'], "patient": patient, "type": jsondata['resourceType'], "target": target, "file": file})
        except:
            print(file, ' failed')
df = pd.DataFrame.from_records(resources)

- Inspect if wanted, filter if wanted, adapt ( df = df[....]) is wanted

In [3]:
df[df.patient == 'Patient/' + patientid]
# df[['type', 'target']].drop_duplicates()
df[df.type == "Medication"]

,id,patient,type,target,file
66,nl-core-AdministrationAgreement-01-Medication-01,None,Medication,subject,Medication-nl-core-AdministrationAgreement-01-...
67,nl-core-AdministrationAgreement-02-Medication-01,None,Medication,subject,Medication-nl-core-AdministrationAgreement-02-...
68,nl-core-AdministrationAgreement-03-Medication-01,None,Medication,subject,Medication-nl-core-AdministrationAgreement-03-...
69,nl-core-AdministrationAgreement-04-Medication-01,None,Medication,subject,Medication-nl-core-AdministrationAgreement-04-...
70,nl-core-DispenseRequest-01-Medication-01,None,Medication,subject,Medication-nl-core-DispenseRequest-01-Medicati...
71,nl-core-MedicationAdministration2-01-Medicatio...,None,Medication,subject,Medication-nl-core-MedicationAdministration2-0...
72,nl-core-MedicationAdministration2-02-Medicatio...,None,Medication,subject,Medication-nl-core-MedicationAdministration2-0...
73,nl-core-MedicationAdministration2-03-Medicatio...,None,Medication,subject,Medication-nl-core-MedicationAdministration2-0...
74,nl-core-MedicationAgreement-01-Medication-01,None,Medication,subject,Medication-nl-core-MedicationAgreement-01-Medi...
75,nl-core-MedicationAgreement-02-Medication-01,None,Medication,subject,Medication-nl-core-MedicationAgreement-02-Medi...


- Define server
- Create patient, see patientid above
- Don't forget to point at the riht file for this patient

In [6]:
resource = "Patient"
headers = {'Content-Type': 'application/json'}

# with open(os.path.join(zibsr4, filename)) as f:
with open(os.path.join(zibs, filename)) as f:
    jsondata = json.load(f)
resp = requests.put(base + "Patient" + "/" + patientid, data=json.dumps(jsondata), headers=headers)
if resp.status_code not in (200, 201):
    for issue in resp.json()["issue"]:
        print(issue)
resp.status_code, resp.json()["id"]

(200, 'nl-core-Patient-01')

- Create a list of resource types to submit, create those
- Take care, resources are dependent on each other and have to be submitted in the right order
- Needs a bit of tweaking and running multiple times, with the right resources submitted each time

In [7]:
types = [
 # 'AllergyIntolerance',
 'Condition',
 # 'Consent',
 # 'Coverage',
 # 'Device',
 # 'DeviceUseStatement',
 # 'Encounter',
 # 'Flag',
 #  'Immunization',
 #  'ImmunizationRecommendation',
 # 'Medication',
 # 'MedicationAdministration',
 # 'MedicationRequest',
 # 'MedicationStatement',
 # 'NutritionOrder',
  # 'Observation',
 #  'Organization',
 #  'Practitioner',
 #  'PractitionerRole',
 #  'Procedure',
 # 'RelatedPerson',
 #   'Specimen'
]
# types = ['Device']
# 'Consent' fails, if we add this then $summary gives an HAPI error.
reslist = df[df.type.isin(types)].to_dict(orient="records")
len(reslist)

16

For each entry in the resource list, create it

In [8]:
success = 0
for res in reslist:
    with open(os.path.join(zibs, res["file"])) as f:
        jsondata = json.load(f)
    create = requests.put(base + res["type"] + "/" + res["id"], data=json.dumps(jsondata), headers=headers)
    if create.status_code not in (200, 201):
        for issue in create.json()["issue"]:
            print(issue)
    else:
        success += 1
        print(create.status_code, res["file"], success)
print('Created ' + str(success) + ' resources')

200 Condition-nl-core-AdvanceDirective-01-Condition-01--30957904.json 1
200 Condition-nl-core-CareTeam-01-Condition-01--40600350.json 2
200 Condition-nl-core-CareTeam-01-Condition-02--64163473.json 3
200 Condition-nl-core-CareTeam-01-Condition-03--59325057.json 4
200 Condition-nl-core-FreedomRestrictingIntervention-01-Condition-01--15337269.json 5
200 Condition-nl-core-LegalSituation-LegalStatus-01--30131813.json 6
200 Condition-nl-core-LegalSituation-Representation-02--25717600.json 7
200 Condition-nl-core-MedicalDevice-01-Condition-01--4085744.json 8
200 Condition-nl-core-MedicalDevice-02-Condition-01--48932206.json 9
200 Condition-nl-core-MedicalDevice-03-Condition-01--7954964.json 10
200 Condition-nl-core-NutritionAdvice-01-Condition-01--3913996.json 11
201 Condition-nl-core-Pregnancy-01--62855489.json 12
200 Condition-nl-core-Procedure-01-Condition-01--26948044.json 13
200 Condition-nl-core-SkinDisorder-01--49507636.json 14
200 Condition-nl-core-SkinDisorder-01-Condition-01--42915

- Get the patient summary, save it
- Show errors if errors

In [11]:
operation = '/$summary'
resp = requests.get(base + 'Patient' + "/" + patientid + operation)
with open('ips-' + patientid + '.json', 'w') as outfile:
    outfile.write(json.dumps(resp.json()))
if create.status_code not in (200, 201):
    for issue in create.json()["issue"]:
        print(issue)
else:
    print(create.status_code)

200


- Inspect the IPS if wanted

In [12]:
JSON(resp.json())

<IPython.core.display.JSON object>

- Make a list with of all resources which are in the type list, get the id's
- Type list is in cell up above

In [ ]:
def delete_res(trash):
    deleted = 0
    for cnt, res in enumerate(trash):
        resp = requests.delete(base + res["type"] + '/' + res["id"])
        if resp.status_code not in (200, 201):
            for issue in resp.json()["issue"]:
                print(issue)
        else:
            deleted += 1
    return deleted

- Delete the resources - may need pultiple runs because of dependencies. May take a while.
- Only needed on servers which don't have cascade delete, see below

In [ ]:
trash = []
# types = ['Consent']
for restype in types:
    query = '?patient=' + patientid
    resp = requests.get(base + restype + query)
    # print(restype, len(resp.json()['entry']) if 'entry' in resp.json() else 0)
    try:
        for entry in resp.json()['entry']:
            trash.append({"type": restype, "id": entry['resource']['id']})
    except KeyError:
        pass
    # For now we just try both 'subject' and 'patient', this could be improved
    query = '?subject=' + patientid
    resp = requests.get(base + restype + query)
    # print(restype, len(resp.json()['entry']) if 'entry' in resp.json() else 0)
    try:
        for entry in resp.json()['entry']:
            trash.append({"type": restype, "id": entry['resource']['id']})
    except KeyError:
        pass
delete_res(trash)

- Delete the patient resource - will fail on some servers if there still are dependent resources, HAPI support cascade delete

In [ ]:
href = base + 'Patient/' + patientid + '?_cascade=delete'
print(href)
resp = requests.delete(href)
if resp.status_code not in (200, 201):
    for issue in resp.json()["issue"]:
        print(issue)
else:
    print(resp.status_code)

# Below are helper routines, don't need to be run in order after the others

- Ad hoc routine
- This cell is just to delete a chosen resource, usually no need to run this

In [17]:
resp = requests.delete(base + "MedicationStatement/nl-core-MedicationUse2-01" + '?_cascade=delete')
# resp = requests.delete(base + "Observation/nl-core-Pregnancy.PregnancyDuration-01")
resp.status_code

200

In [18]:
JSON(resp.json())

<IPython.core.display.JSON object>

- Ad hoc routine
- Routine to remove meta, which contains the profile, which will not be known internationally anyway. Run before submitting resources, needs to be run only once.

In [ ]:
for file in files:
    if file[0] == '.':
        continue
    with open(os.path.join(zibs, file)) as f:
        jsondata = json.load(f)
        try:
            print(jsondata['subject'])
        except:
            print('no subject')
    #    jsondata.pop('meta', 'no meta')
    # with open(os.path.join(zibsr4, file), 'w') as f:
    #     json.dump(jsondata, f)

In [ ]:
for file in files:
    if file[0] == '.':
        continue
    with open(os.path.join(zibs, file)) as f:
        jsondata = json.load(f)
        try:
            print(jsondata['subject'])
        except:
            print('no subject')
    #    jsondata.pop('meta', 'no meta')
    # with open(os.path.join(zibsr4, file), 'w') as f:
    #     json.dump(jsondata, f)